# DIVALGO demonstration

This notebook contains a demostration of the DIVALGO tool

In [1]:
import streamlit as st
import importlib
import divalgo.util
importlib.reload(divalgo.util)
import divalgo.util as div
import os
import sklearn.linear_model as lm
import numpy as np
from PIL import Image
from sklearn import metrics

## Actual data - dogs and wolfs

In [2]:
# Load data
dogs = sorted(os.listdir(os.path.join("..", "data", "data", "dogs")))
wolves =  sorted(os.listdir(os.path.join("..", "data", "data", "wolves")))


In [3]:
# Preprocessing
img_size = 50
dogs_images = []
wolves_images = [] 

for i in dogs:
    if os.path.isfile(os.path.join("..", "data", "data", "dogs", f"{i}")):
        img = Image.open(os.path.join("..", "data", "data", "dogs", f"{i}")).convert('L')            
        img = img.resize((img_size,img_size), Image.ANTIALIAS)
        img = np.asarray(img)/255.0
        dogs_images.append(img)    

for i in wolves:
    if os.path.isfile(os.path.join("..", "data", "data", "wolves", f"{i}")):
        img = Image.open(os.path.join("..", "data", "data", "wolves", f"{i}")).convert('L')
        img = img.resize((img_size,img_size), Image.ANTIALIAS)
        img = np.asarray(img)/255.0            
        wolves_images.append(img)     

C:\Users\stnyl\AppData\Local\Temp\ipykernel_11040\684101750.py:9: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((img_size,img_size), Image.ANTIALIAS)
C:\Users\stnyl\AppData\Local\Temp\ipykernel_11040\684101750.py:16: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((img_size,img_size), Image.ANTIALIAS)


In [4]:
# Manual train-test split (to track filenames)
X_train = np.asarray(dogs_images[0:800] + wolves_images[0:800])
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])
X_test = np.asarray(dogs_images[800:1000] + wolves_images[800:1000])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])
y_train = np.asarray(["dog" for y in range(800)] + ["wolf" for y in range(800)])
y_train = y_train.reshape(y_train.shape[0],1)
y_test_ar = np.asarray(["dog" for y in range(200)] + ["wolf" for y in range(200)])
y_test = y_test_ar.reshape(y_test_ar.shape[0],1)

y_train, y_test = [k.T for k in [y_train, y_test]]
filenames_test = [os.path.join("..", "data", "data", "dogs", d) for d in dogs[800:1000]] + [os.path.join( "..", "data", "data", "wolves", w) for w in wolves[800:1000]]

In [5]:
# Train model
model = lm.LogisticRegression(penalty='none', tol=0.1).fit(X_train, y_train[0])

c:\Users\stnyl\anaconda3\envs\exam_hci\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
y_pred = model.predict(X_test)
dog_wolf = div.Evaluate((X_test, y_test[0], y_pred, filenames_test), model)

In [8]:
dog_wolf.open_visualization()

Training: 

Train - epoch - loss over time, learning curves (Frida)

Coefficient heatmap (Jakob)

Visualize images (correct/wrong predictions) - Stine 

In [16]:
import pandas as pd
d = pd.DataFrame()

In [17]:
d["y_test"] = y_test[0]

In [20]:
d["correct_prediction"] = y_test[0] == y_pred
d["false_prediction"] = y_test[0] != y_pred

In [41]:
df = pd.DataFrame(X_test)
df["y_test"] = y_test[0]
df["y_pred"] = y_pred

In [ ]:
df["correct_prediction"] = df["y_test"] == df["y_pred"]
df["filename"] = filenames_test

In [50]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,2494,2495,2496,2497,2498,2499,y_test,y_pred,correct_prediction,filename
0,0.400000,0.407843,0.196078,0.219608,0.258824,0.200000,0.176471,0.231373,0.270588,0.243137,...,0.372549,0.403922,0.403922,0.482353,0.462745,0.388235,dog,wolf,False,..\data\data\dogs\n02109525_17140.jpg
1,0.572549,0.552941,0.466667,0.360784,0.415686,0.584314,0.647059,0.654902,0.674510,0.690196,...,0.513725,0.517647,0.529412,0.509804,0.509804,0.509804,dog,dog,True,..\data\data\dogs\n02109525_1741.jpg
2,0.160784,0.176471,0.172549,0.164706,0.156863,0.164706,0.164706,0.160784,0.141176,0.125490,...,0.639216,0.564706,0.486275,0.423529,0.517647,0.600000,dog,dog,True,..\data\data\dogs\n02109525_18371.jpg
3,0.929412,0.909804,0.878431,0.890196,0.847059,0.686275,0.384314,0.203922,0.333333,0.490196,...,0.333333,0.215686,0.200000,0.180392,0.192157,0.203922,dog,wolf,False,..\data\data\dogs\n02109525_5013.jpg
4,0.305882,0.247059,0.278431,0.262745,0.243137,0.254902,0.333333,0.301961,0.321569,0.317647,...,0.788235,0.819608,0.823529,0.780392,0.729412,0.811765,dog,wolf,False,..\data\data\dogs\n02109525_9522.jpg


In [51]:
df.sample(6)

,0,1,2,3,4,5,6,7,8,9,...,2494,2495,2496,2497,2498,2499,y_test,y_pred,correct_prediction,filename
253,0.894118,0.949020,0.968627,0.980392,0.992157,0.992157,0.992157,0.992157,0.929412,0.972549,...,0.427451,0.345098,0.282353,0.258824,0.254902,0.258824,wolf,wolf,True,..\data\data\wolves\wolf-1583207__480.jpg
144,0.184314,0.203922,0.317647,0.156863,0.200000,0.156863,0.235294,0.215686,0.243137,0.168627,...,0.207843,0.180392,0.180392,0.196078,0.207843,0.274510,dog,wolf,False,..\data\data\dogs\n02113186_5340.jpg
300,0.137255,0.094118,0.058824,0.066667,0.078431,0.078431,0.074510,0.074510,0.078431,0.141176,...,0.200000,0.192157,0.196078,0.176471,0.176471,0.231373,wolf,dog,False,..\data\data\wolves\wolf-2878890__480.jpg
390,0.760784,0.764706,0.772549,0.780392,0.788235,0.792157,0.796078,0.800000,0.803922,0.811765,...,0.607843,0.654902,0.627451,0.619608,0.725490,0.627451,wolf,dog,False,..\data\data\wolves\wolf-in-sheeps-clothing-25...
188,0.576471,0.580392,0.603922,0.552941,0.576471,0.603922,0.627451,0.654902,0.698039,0.694118,...,0.129412,0.117647,0.117647,0.109804,0.098039,0.082353,dog,wolf,False,..\data\data\dogs\n02115913_3863.jpg
168,0.713725,0.749020,0.733333,0.745098,0.776471,0.796078,0.823529,0.823529,0.768627,0.815686,...,0.737255,0.709804,0.686275,0.713725,0.756863,0.752941,dog,wolf,False,..\data\data\dogs\n02113799_447.jpg
